Semi-inducibility
=================

First cell sets up the theory of colored graphs and provides a script to translate semi-inducibility problems to an optimization over regular graphs. 
Then this is used to maximize the semi-inducibility of the alternating $P_4$ and the alternating $C_6$.

In [9]:
G = GraphTheory

G.printlevel(0)

def check_blocks(flg, do, dont):
    eds = flg.blocks("edges")
    for ee in do:
        if ee not in eds:
            return False
    for ee in dont:
        if ee in eds:
            return False
    return True

AG = Theory("BlueGraph", "blue_edges")
CG = combine("RedBlueGraph", G, AG, symmetries=FullSymmetry)
CG.exclude(CG(2, edges=[[0, 1]], blue_edges=[[0, 1]]))

def get_targ(ff):
    vn = ff.size()
    eddo = ff.blocks("edges")
    eddont = ff.blocks("blue_edges")
    pats = G.p(vn, edges=eddo, edges_m=eddont)
    targ = 0
    for xx in pats.compatible_flags():
        coef = 0
        for eperm in itertools.permutations(range(vn)):
            xxpermed = xx.subflag(eperm)
            if check_blocks(xxpermed, eddo, eddont):
                coef += 1
        targ += coef*xx
    return targ*QQ(1/(factorial(vn)))

In [10]:
#
# Calculation for alternating P4
#

targ = get_targ(CG(4, edges=[[1, 2]], blue_edges=[[0, 1], [2, 3]]))
constr = G.blowup_construction(4, 1, edges={(0, 0): 1/3})
dens = constr.density(targ)
bound = G.optimize(targ, 4, construction=constr, exact=True, file="certificates/semiind_p4")
print("For the alternating P4 the constr gives {}, upper bound is {}".format(dens, bound))


#
# Calculation for alternating C6
#

targ = get_targ(CG(6, edges=[[0, 1], [2, 3], [4, 5]], blue_edges=[[1, 2], [3, 4], [5, 0]]))
constr = G.blowup_construction(6, 1, edges={(0, 0): 1/2})
dens = constr.density(targ)
bound = G.optimize(targ, 6, construction=constr, exact=True, file="certificates/semiind_c6")
print("For the alternating C6 the constr gives {}, upper bound is {}".format(dens, bound))

For the alternating P4 the constr gives 4/27, upper bound is 4/27
For the alternating C6 the constr gives 1/64, upper bound is 1/64


Tools to test stability
====================

First a script following Corollary 2.2 is included, which will test regularity for P4 and C6 in the following cell.

In [11]:
from fractions import Fraction
from sage.algebras.combinatorial_theory import _unflatten_matrix
import pickle

# general helper function to have data in sage compatible format
def to_sage(dim, data):
    if dim==0:
        if isinstance(data, Fraction):
            return QQ(data)
        if isinstance(data, float):
            return RR(data)
        return data
    return [to_sage(dim-1, xx) for xx in data]

def check_almost_regularity(name):
    file = "certificates/semiind_"+str(name)
    print("\n\nchecking almost regularity for " + file)
    G.reset()
    G.printlevel(0)
    if not file.endswith(".pickle"):
        file += ".pickle"
    with open(file, "rb") as f:
        certificate = pickle.load(f)
    target_size = certificate["target size"]
    edge_type = G(2, edges=[[0, 1]], ftype=[0, 1])
    edge_regularity = G(3, edges=[[0, 1], [0, 2]], ftype=[0, 1]) - G(3, edges=[[0, 1], [1, 2]], ftype=[0, 1])
    nonedge_type = G(2, edges=[], ftype=[0, 1])
    nonedge_regularity = G(3, edges=[[0, 2]], ftype=[0, 1]) - G(3, edges=[[1, 2]], ftype=[0, 1])
    
    for ii,xx in enumerate(certificate["typed flags"]):
        fsz = xx[0]
        ftp = xx[1]
        if ftp == edge_type._pythonize():
            X = matrix(to_sage(2, _unflatten_matrix(certificate["X matrices"][ii])[0]))
            reg_vec = (edge_regularity<<(fsz - 3)).values()
            if reg_vec in X.image():
                print(" - almost degree regularity holds between edges")
            else:
                print(" - almost degree regularity fails between edges")
        elif ftp == nonedge_type._pythonize():
            X = matrix(to_sage(2, _unflatten_matrix(certificate["X matrices"][ii])[0]))
            reg_vec = (nonedge_regularity<<(fsz - 3)).values()
            if reg_vec in X.image():
                print(" - almost degree regularity holds between nonedges")
            else:
                print(" - almost degree regularity fails between nonedges")

In [12]:
# Almost regularity checks based on Corollary 2.2
check_almost_regularity("p4")
check_almost_regularity("c6")



checking almost regularity for certificates/semiind_p4
 - almost degree regularity holds between nonedges
 - almost degree regularity holds between edges


checking almost regularity for certificates/semiind_c6
 - almost degree regularity holds between nonedges
 - almost degree regularity holds between edges
